# Step 3: Data Preprocessing
This notebook handles the preprocessing stage of the diabetes prediction pipeline.
It includes handling missing values, feature engineering, encoding, feature selection,
and artifact logging to Weights & Biases.

In [1]:
!pip install -q wandb


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import wandb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 1. Load data

In [3]:
def load_raw_data_from_wandb(project, artifact_path,entity='ngocnhi-p4work-national-economics-university'):
    wandb.login()
    run = wandb.init(project=project,entity=entity, job_type='process_data')
    artifact = run.use_artifact(artifact_path)
    artifact_dir = artifact.download()
    df = pd.read_csv(os.path.join(artifact_dir, 'raw_data.csv'))
    return df, run

## 2. Feature engineering pipeline

### 2.1 Handle columns with unreasonable zero values

In [4]:
# Get columns with 0
def get_zero_columns(df):
    return [col for col in df.columns if df[col].min() == 0 and col not in ['Pregnancies', 'Outcome']]

# Analyze the relationship between columns with missing values and the Outcome column
def analyze_missing_impact(df, target, na_columns, threshold=0.1):
    """
    Analyze the impact of missing values on the target variable.
    Returns 2 list:
    - columns_to_flag: Columns to keep NA_FLAG
    - columns_to_fill: Columns that only need to be filled with median
    """
    columns_to_flag = []
    columns_to_fill = []
    temp_df = df.copy()
    for col in na_columns:
        temp_df[col + '_NA_FLAG'] = temp_df[col].isnull().astype(int)
        summary = temp_df.groupby(col + '_NA_FLAG')[target].mean()
        if len(summary) == 2:  # Ensure both groups 0 and 1 are present
            diff = abs(summary[0] - summary[1])
            if diff >= threshold:
                columns_to_flag.append(col)
            else:
                columns_to_fill.append(col)
        else:
            columns_to_fill.append(col)  # If there is only one group, it will still fill.
    return columns_to_flag, columns_to_fill

# Handling columns with missing values
def process_missing_values(df, columns_to_flag, columns_to_fill):
    """
    Handling missing values:
    - For columns_to_flag: create an additional column _NA_FLAG
    - For columns_to_fill: fill with the median value
    """
    for col in columns_to_flag:
        df[col + '_NA_FLAG'] = df[col].isnull().astype(int)
        df[col].fillna(df[col].median(), inplace=True) # keep fill median
    for col in columns_to_fill:
        df[col].fillna(df[col].median(), inplace=True)
    return df

### 2.2 Create new features 

In [5]:
def create_features(df):
    # 1, Creating a new age variable by dividing the age variable into categories.
    df.loc[(df['Age'] >= 21) & (df['Age'] < 50), 'NEW_AGE_CAT'] = 'mature'
    df.loc[(df['Age'] >= 50), 'NEW_AGE_CAT'] = 'senior'
    # 2, BMI below 18.5 is underweight, 18.5 to 24.9 is normal, 24.9 to 29.9 is Overweight, and over 30 is obese
    df['NEW_BMI'] = pd.cut(df['BMI'], bins=[0, 18.5, 24.9, 29.9, 100], labels=['Underweight', 'Healthy', 'Overweight', 'Obese'])
    # 3,Converting glucose value to categorical variable
    df['NEW_GLUCOSE'] = pd.cut(df['Glucose'], bins=[0, 140, 200, 300], labels=['Normal', 'Prediabetes', 'Diabetes'])
    # 4, Creating a categorical variable by considering age and body mass index together, 3 breakdowns were caught
    df['NEW_INSULIN_SCORE'] = df['Insulin'].apply(lambda x: 'Normal' if 16 <= x <= 166 else 'Abnormal')
    # 5, Creating a categorical variable by considering age and glucose values together
    df['NEW_GLUCOSE*INSULIN'] = df['Glucose'] * df['Insulin']
    # 6, Deriving Categorical Variable with Insulin Value
    df['NEW_GLUCOSE*PREGNANCIES'] = df['Glucose'] * df['Pregnancies']
    return df

### 2.3 Encoding

In [6]:
def encode_features(df):
    # 1, The process of separating variables according to their types.
    def grab_col_names(df, cat_th=10, car_th=20):
        # cat_cols, cat_but_car
        cat_cols = [col for col in df.columns if df[col].dtypes == "O"]
        num_but_cat = [col for col in df.columns if df[col].nunique() < cat_th and df[col].dtypes != "O"]
        cat_but_car = [col for col in df.columns if df[col].nunique() > car_th and df[col].dtypes == "O"]
        cat_cols = cat_cols + num_but_cat
        cat_cols = [col for col in cat_cols if col not in cat_but_car]
        # num_cols
        num_cols = [col for col in df.columns if df[col].dtypes != "O"]
        num_cols = [col for col in num_cols if col not in num_but_cat]
        return cat_cols, num_cols, cat_but_car
    cat_cols, num_cols, cat_but_car = grab_col_names(df)
    #2, Label encoding for binary categorical variables
    binary_cols = [col for col in df.columns if df[col].dtypes == "O" and df[col].nunique() == 2]
    for col in binary_cols:
        df[col] = LabelEncoder().fit_transform(df[col])
    #3, One-hot encoding for categorical variables with more than 2 categories
    cat_cols = [col for col in cat_cols if col not in binary_cols and col not in ["Outcome"]]
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    return df

### 2.4 Feature Selection

In [7]:
def select_features_mi(df, target_col='Outcome', k=8):
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_df = pd.DataFrame({'Feature': X.columns, 'MI': mi_scores}).sort_values(by='MI', ascending=False)
    top_features = mi_df['Feature'].head(k).tolist()
    return df[top_features + [target_col]]

### 2.5 Convert the columns to uppercase

In [8]:
def uppercase_column_names(df):
    """
    Convert all column names of the DataFrame to uppercase.
    """
    df.columns = [col.upper() for col in df.columns]
    return df

### 2.5 Log artifact to WandB

In [9]:
def save_and_log_artifact(df, filename, run, artifact_name, artifact_type, description):
    df.to_csv(filename, index=False)
    artifact = wandb.Artifact(name=artifact_name, type=artifact_type, description=description)
    artifact.add_file(filename)
    run.log_artifact(artifact)
    run.finish()
    print(f"Saved and logged {filename} to W&B")

In [10]:
# === RUN PIPELINE ===
project = 'diabetes'
artifact_path = 'ngocnhi-p4work-national-economics-university/diabetes/raw_data.csv:latest'
df, run = load_raw_data_from_wandb(project, artifact_path)

# Mark 0s as NaN
zero_cols = get_zero_columns(df)
df[zero_cols] = df[zero_cols].replace(0, np.nan)

# Handle missing
to_flag, to_fill = analyze_missing_impact(df, 'Outcome', zero_cols)
df = process_missing_values(df, to_flag, to_fill)

# Feature engineering
df = create_features(df)

# Encoding
df = encode_features(df)

# Feature selection
df = select_features_mi(df, 'Outcome', k=8)
df.drop_duplicates(inplace=True)

# Uppercase column names
df = uppercase_column_names(df)

# Save + log
save_and_log_artifact(df, 'preprocessed_data.csv', run, 'preprocessed_data.csv', 'clean_data', 'Cleaned and processed diabetes data')


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlcolongmay (mlcolongmay-neu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


Saved and logged preprocessed_data.csv to W&B


In [11]:
df

,GLUCOSE,NEW_GLUCOSE*INSULIN,BMI,NEW_GLUCOSE_PREDIABETES,NEW_BMI_OBESE,AGE,NEW_GLUCOSE*PREGNANCIES,NEW_BMI_HEALTHY,OUTCOME
0,148.0,18500.0,33.6,True,True,50,888.0,False,1
1,85.0,10625.0,26.6,False,False,31,85.0,False,0
2,183.0,22875.0,23.3,True,False,32,1464.0,True,1
3,89.0,8366.0,28.1,False,False,21,89.0,False,0
4,137.0,23016.0,43.1,False,True,33,0.0,False,1
...,...,...,...,...,...,...,...,...,...
763,101.0,18180.0,32.9,False,True,63,1010.0,False,0
764,122.0,15250.0,36.8,False,True,27,244.0,False,0
765,121.0,13552.0,26.2,False,False,30,605.0,False,0
766,126.0,15750.0,30.1,False,True,47,126.0,False,1
